In [1]:
import requests
import json
from bs4 import BeautifulSoup
from selenium import webdriver

# Getting the Site ID

In [6]:
response = requests.get('https://api.dineoncampus.com/v1/sites/uchicago/info')
data = response.json()
site_id = data['site']['id']
print("Site ID:", site_id)

Site ID: 60648db131d1ee201bfa84a8


# Info by Slug

In [8]:
slug_request_url = 'https://api.dineoncampus.com/v1/sections/info_by_slug?site_id={}&slug=whats-on-the-menu'.format(site_id)
response = requests.get(slug_request_url)
data = response.json()
print(data)

{'status': 'success', 'request_time': 0.01780681, 'records': 1, 'section': {'id': '60648db431d1ee201bfa84b1', 'name': "What's On The Menu?", 'nodes': None, 'body': 'null', 'show_header': True, 'is_url': False, 'url': None, 'published': True, 'start': '2021-07-21T12:29:45.000-07:00', 'end': '2021-07-21T12:29:45.000-07:00', 'scheduled': False, 'is_file': False, 'file_name': None, 'original_file_name': None, 'global': False, 'change_name': False, 'change_slug': False, 'master': False, 'slug': 'whats-on-the-menu', 'use_banner': False, 'microsite_landing': False, 'banner_text': 'null', 'banner_text_align': 'left', 'banner_text_font': 'inherit', 'tile': False, 'is_published': True, 'no_robots': False, 'script_url': None, 'use_script': False, 'published_status': None, 'master_section': {'id': '5956bc3fee596f675f69f653', 'name': "What's On The Menu?", 'master_slug': 'whats-on-the-menu'}, 'banner': None}}


# All Locations

In [17]:
def building_of_interest(building_name):
    return building_name == 'Bartlett Dining Commons' or building_name == 'Cathey Dining Commons' or building_name == 'Baker Dining Commons' or building_name == 'Woodlawn Dining Commons'

In [53]:
def convert_tuples_to_dict(lst):
    result = {}
    for tuple_item in lst:
        key = tuple_item[1]
        value = tuple_item[0]
        result[key] = value
    return result


In [54]:
all_location_url = 'https://api.dineoncampus.com/v1/locations/all_locations?platform=0&site_id={}&for_menus=true&with_address=false&with_buildings=true'.format(site_id)
response = requests.get(all_location_url)
data = response.json()
# Extract Building IDs
building_ids = []
for building in data['buildings']:
    for location in building['locations']:

        if building_of_interest(location['name']) and location['active']:
            building_ids.append((location['id'], location['name']))

building_ids = convert_tuples_to_dict(building_ids)
print(building_ids)


{'Baker Dining Commons': '618a6caab63f1e2d4442bdf5', 'Cathey Dining Commons': '618a6efbb63f1e2d444389c1', 'Woodlawn Dining Commons': '618a6df9b63f1e2d692b1f5c', 'Bartlett Dining Commons': '618a6f95b63f1e2d3b454065'}


# Periods

In [44]:
response_list = []
for building_id, building_name in building_ids:
    period_request_url = 'https://api.dineoncampus.com/v1/location/{}/periods?platform=0&date=2023-5-18'.format(building_id)
    response = requests.get(period_request_url)
    data = response.json()
    if not data['closed']:
        response_list.append((building_name, data))


In [51]:
print(response_list[0][0], "," ,response_list[0][1]['periods'])
print(response_list[0][0], "," , response_list[1][1]['periods'])
print(response_list[0][0], "," ,response_list[2][1]['periods'])

    

Baker Dining Commons , [{'id': '64663e52351d53059a4d66d6', 'name': 'Breakfast', 'sort_order': 0}, {'id': '64663e52351d53059a4d66db', 'name': 'Lunch', 'sort_order': 1}, {'id': '64663e52351d53059a4d66ec', 'name': 'Dinner', 'sort_order': 2}, {'id': '64663e53351d53059a4d6700', 'name': 'Salad Bar', 'sort_order': 3}, {'id': '64663e53351d53059a4d670a', 'name': 'Deli Bar', 'sort_order': 4}, {'id': '64663e53351d53059a4d6705', 'name': 'Every Day', 'sort_order': 5}, {'id': '64663e53351d53059a4d66fc', 'name': 'Late Night', 'sort_order': 6}]
Baker Dining Commons , [{'id': '646643d1351d5305ae38f613', 'name': 'Breakfast', 'sort_order': 0}, {'id': '646643d0351d5305ae38f5f9', 'name': 'Lunch', 'sort_order': 1}, {'id': '646643d1351d5305ae38f607', 'name': 'Dinner', 'sort_order': 2}, {'id': '646643d1351d5305ae38f618', 'name': 'Salad Bar', 'sort_order': 3}, {'id': '646643d1351d5305ae38f622', 'name': 'Deli Bar', 'sort_order': 4}, {'id': '646643d1351d5305ae38f61d', 'name': 'Every Day', 'sort_order': 5}, {'id'

In [59]:
master_dictionary = {}
for building in response_list:
    name = building[0]
    master_dictionary[name] = {}
    for period in building[1]['periods']:
        if period['name'] == 'Lunch' or period['name'] == 'Dinner':
            master_dictionary[name][period['name']] = period['id']
        master_dictionary[name]['location_id'] = building_ids[name]

In [60]:
print(master_dictionary)

{'Baker Dining Commons': {'location_id': '618a6caab63f1e2d4442bdf5', 'Lunch': '64663e52351d53059a4d66db', 'Dinner': '64663e52351d53059a4d66ec'}, 'Cathey Dining Commons': {'location_id': '618a6efbb63f1e2d444389c1', 'Lunch': '646643d0351d5305ae38f5f9', 'Dinner': '646643d1351d5305ae38f607'}, 'Woodlawn Dining Commons': {'location_id': '618a6df9b63f1e2d692b1f5c', 'Lunch': '6466421ac625af0654cd1b05', 'Dinner': '6466421bc625af0654cd1b20'}}
